In [1]:
import pandas as pd
from collections import defaultdict

In [2]:
processed_bids = pd.read_pickle("../outputs/processed_bids.pkl")

In [3]:
new_table = defaultdict(dict)
for idx, row in processed_bids.iterrows():
    id = row["GEOID"]
    bidder = row["bidder"]
    if "bidders" not in new_table[id]:
        new_table[id]["bidders"] = dict()
    new_table[id]["all_neighbors"] = row["all_neighbors"]
    new_table[id]["geometry"] = row["geometry"]
    new_table[id]["bidders"][bidder] = {"neighbors": row["neighbors"], "round": row["round"],
                                        "neighbor_status": row["neighbor_status"]}
    if row["winner"]:
        new_table[id]["winner"] = bidder
        new_table[id]["blocks"] = row["won_blocks"]
        new_table[id]["round"] = row["round"]

In [12]:
collapsed_bids = pd.DataFrame.from_dict(new_table , orient='index')
collapsed_bids = collapsed_bids.reset_index().rename(columns={"index":"GEOID"})
collapsed_bids

,GEOID,bidders,all_neighbors,geometry,winner,blocks,round
0,10010207001,"{'1stel, Inc.': {'neighbors': [], 'round': 9, ...","{10010202002: {('WideOpenWest Finance, LLC', '...","POLYGON ((-86.47038099999999 32.456581, -86.47...",Space Exploration Technologies Corp.,"[10010207001071, 10010207001073, 10010207001095]",13.0
1,10010208011,"{'Hughes Network Systems, LLC': {'neighbors': ...","{10010206002: {('WideOpenWest Finance, LLC', '...","POLYGON ((-86.59549799999999 32.381123, -86.59...",Space Exploration Technologies Corp.,"[10010208011010, 10010208011017, 1001020801102...",13.0
2,10010208012,"{'Hughes Network Systems, LLC': {'neighbors': ...","{10010201001: {('WideOpenWest Finance, LLC', '...","POLYGON ((-86.591984 32.44164, -86.591976 32.4...",Space Exploration Technologies Corp.,"[10010208012004, 10010208012010, 1001020801201...",13.0
3,10010208021,"{'Hughes Network Systems, LLC': {'neighbors': ...","{10010201001: {('WideOpenWest Finance, LLC', '...","POLYGON ((-86.645787 32.554375, -86.6457779999...",RDOF USA Consortium,"[10010208021003, 10010208021006, 1001020802101...",13.0
4,10010209001,"{'Hughes Network Systems, LLC': {'neighbors': ...","{10010208021: {('Central Access, Inc', 'Centra...","POLYGON ((-86.664761 32.627948, -86.6647069999...",RDOF USA Consortium,"[10010209001000, 10010209001001, 1001020900100...",13.0
...,...,...,...,...,...,...,...
61736,560399676003,{'AMG Technology Investment Group LLC': {'neig...,"{160199701001: {('Cable One, Inc.', 'Cable ONE...","POLYGON ((-111.045492 43.51067099999999, -111....",Space Exploration Technologies Corp.,"[560399676003075, 560399676003087, 56039967600...",13.0
61737,560399677021,{'AMG Technology Investment Group LLC': {'neig...,"{560399676001: {('Rise Broadband', 'Skybeam, L...","POLYGON ((-111.039007 43.472801, -111.036624 4...",Space Exploration Technologies Corp.,"[560399677021047, 560399677021058, 56039967702...",13.0
61738,560419753001,{'AMG Technology Investment Group LLC': {'neig...,{490439641011: {('COMCAST CABLE COMMUNICATIONS...,"POLYGON ((-111.046818 41.156019, -111.046818 4...",Space Exploration Technologies Corp.,[560419753001107],13.0
61739,560430002001,{'AMG Technology Investment Group LLC': {'neig...,{560039626002: {('Mountain West Technologies C...,"POLYGON ((-108.550788 44.12770099999999, -108....",Space Exploration Technologies Corp.,"[560430002001181, 560430002001513]",13.0


In [5]:
bids = pd.read_csv("../data/all_bids.csv")
bidders = bids.groupby("bidder").agg({"t+l_weight":"first", "tier":"first", "latency":"first"}).reset_index()
name_map = {
    "CCO Holdings, LLC":"Charter Communications Inc",
    "Windstream Services LLC, Debtor-In-Possession":"Windstream Holdings, Inc.",
    "Connect Everyone LLC":"Starry, Inc.",
    "Frontier Communications Corporation, DIP":"Frontier Communications Corporation",
    "California Internet, L.P. dba GeoLinks":"GeoLinks",
    "Etheric Communications LLC":"Etheric Networks, Inc.",
    "Consolidated  Communications, Inc.":"Consolidated Communications, Inc.",
    "Frontier Communications Northwest, LLC":"Ziply Fiber",
    "Citynet West Virginia, LLC":"Citynet, LLC",
    "Computer 5, Inc. d/b/a LocalTel Communications": "LocalTel Communications",
    "Armstrong Telephone Company - Northern Division": "Armstrong Holdings, Inc.",
    "Commnet Wireless, LLC": "ATN International, Inc.",
    "Mediacom Communications Corporation": "Mediacom Communications Corp.",
    "Continental Divide Electric Cooperative": "Continental Divide Electric Cooperative, Inc.",
    "South Arkansas Telephone Company": "South Arkansas Telephone Co.",
    "Digital Connections Inc. dba PRODIGI": "Digital Connections, Inc.",
    "Shenandoah Cable Television, LLC": "Shenandoah Telecommunications Company",
    "Direct Communications Rockland, Inc": "Direct Communications Rockland, Inc.",
    "Pine Belt Communications, Inc.":"Pine Belt Communications Co. Inc.",
    "Savage Communications":"Savage Communications Inc.",
    "Hamilton County Telephone Co-op":"Hamilton County Telephone Co-Op",
    "Hotwire Communications, Ltd":"Hotwire Communications Ltd.",
    "Altice USA, Inc.":"Altice",
    "TruVista Communications, Inc.":"TruVista Communications of Georgia, LLC",
    "Hawaii Dialogix Telecom LLC":"Hawaii Dialogix Telecom LLC",
    "Safelink Internet LLC":"Safelink Internet",
    "American Heartland":"Farmers Telephone Company",
    "Cellular Services LLC.":"Cellular Services LLC",
    "Scott County Telephone Cooperative, Inc.":"Scott County",
    "Micrologic Inc.":"Micrologic, Inc.",
    "Pioneer Wireless, Inc":"Pioneer Wireless, Inc.",
    "St. John Telco":"St. John Telephone, Inc.",
    "One Ring Networks, Inc":"One Ring Networks, Inc.",
    "Emery Telephone dba Emery Telcom":"Emery Telcom",
    "XIT Telecommunication & Technology":"XIT Rural Telephone Cooperative, Inc.",
    "Somerset Telephone Co., Inc.":"SOMERSET TELEPHONE COMPANY",
    "Siuslaw Broadband, LLC dba Hyak Technologies":"Siuslaw Broadband, LLC",
    "Minnesota Connections c/o Consolidated Tel Company":"Consolidated Telephone Company",
    "DoCoMo Pacific, Inc.":"Docomo Pacific",
    "Roseau Electric Cooperative, Inc.":"Roseau Electric Cooperative",
    "Custer Telephone Cooperative, Inc.":"Custer Telephone Cooperative Inc.",
    "Lakeland Communications Group, LLC":"Lakeland Communications, Inc.",
    "LigTel Communications, Inc.":"Ligtel Communications",
    "Hamilton Long Distance Company":"Hamilton.net, Inc.",
    "Allen's T.V. Cable Service, Inc.":"Allen's TV Cable Service, Inc.",
    "W. T. Services, Inc.":"W.T. Services, Inc.",
    "Heart of the Catskills Comm. Inc., dba MTC Cable":"MTC Cable",
    "Citizens Vermont Acquisition Corporation":"Citizens Telephone Company",
    "Nova Cablevision, Inc":"Nova Cablevision, Inc.",
    "yondoo Broadband LLC":"yondoo Broadband, LLC",
    "Baraga Telephone Company":"Baraga Telephone Company Inc.",
    "Miles Communications LLC":"Miles Communications, Inc.",
    "PVT NetWorks, Inc.":"Penasco Valley Telephone Cooperative, Inc.",
    "Yucca Telecommunications Systems, Inc.":"Yucca Telecom",
    "H&B Communication's, Inc.":"H&B Enterprises Inc.",
    "MARQUETTE-ADAMS TELEPHONE COOPERATIVE, INC.":"Marquette-Adams Telephone Cooperative, Inc.",
    "Pinpoint Bidding Coalition":"Pinpoint Holdings, Inc.",
    "Computer Techniques, Inc. dba CTI Fiber":"CTI Fiber",
    "Barry Technology Services, LLC":"Barry Technology Services",
    "Bloosurf, LLC":"Bloosurf",
    "St Paul Cooperative Telephone Association":"St Paul Coop Telephone Assoc",
    "Farmers Mutual Cooperative Telephone Company":"FARMERS MUTUAL TELEPHONE COMPANY",
    "NTS Communications, LLC":"NTS, Inc.",
    "Unified Communications Inc.":"Unified Communications, Inc.",
    "Peoples Communication, LLC.":"Peoples Communication, Inc.",
    "Wood County Telephone Company d/b/a Solarus":"Solarus",
    "Comcell Inc.":"Comcell, Inc.",
    "Taylor Telephone Coop., Inc. dba Taylor Telecom":"Taylor Telephone Cooperative, Inc.",
    "Plateau Telecommunications, Incorporated":"Plateau",
    "AMA Communications, LLC":"AMA Communicaitons, L.L.C.",
    "AirCell, Inc.":"AirCell",
    "Bays-ET Highspeed LLC":"Bays-ET Highspeed Internet LLC",
    "Big Bend Telecom LTD":"Big Bend Telephone Company, Inc.",
    "Blue Ridge Cable Technologies, Inc.":"Blue Ridge Cable Technologies, Inc.",
    "Cass Cable TV, Inc.":"Cass Cable TV, Inc.",
    "City of Wilson GreenLight":"City of Wilson",
    "Climax Telephone Company dba CTS Telecom, Inc.":"Climax Telephone Company",
    "Cumberland Telephone Company, Inc":"CUMBERLAND TELEPHONE COMPANY",
    "Cumby Telephone Coooertive, Inc.":"Cumby Telephone Coop., Inc.",
    "Data Stream Mobile Technoligies Inc.":"Data Stream",
    "Get Wireless Inc":"Getwireless.net",
    "Hartington Telecommunications Co., Inc,":"Hartington Telecommunications Co., Inc.",
    "Hilliary Communications Consortium":"Hilliary Communications, LLC",
    "Hillsboro Telephone Company,  Inc.":"Hillsboro Telephone Company",
    "Home Telephone ILEC, LLC":"Home Telephone ILEC, LLC d/b/a Home Telecom",
    "Hood Canal Telephone Co., Inc":"Hood Canal Communications",
    "Kingdom Telecommunications, Inc.":"Kingdom Telecommunications Inc",
    "Lincolnville Communications":"Lincolnville Telephone Company",
    "Local Internet Service Company, Inc.":"Local Internet Service Company",
    "Martell Enterprise  Inc.":"Martell Enterprises, Inc.",
    "Massena Telephone Company, Inc.":"Massena Telephone Company",
    "Moundridge Telephone Company":"Moundridge Telephone Co.",
    "MyServer.org, Inc dba San Diego Broadband":"San Diego Broadband",
    "Newbreak Management, LLC":"Newbreak Communications",
    "NexGenAccess Incorporated":"NexGenAccess",
    "Northwoods Communication Technologies, LLC":"Northwoodsconnect",
    "One Point Technologies Inc":"One Point Technologies Inc.",
    "Panhandle Telecommunication Systems, Inc.":"Panhandle Telephone Cooperative, Inc.",
    "Pathwayz Communications Inc":"Pathwayz Communications, Inc.",
    "RONAN TELEPHONE COMPANY":"Ronan Telephone Co",
    "Rainbow Communications LLC":"Rainbow Telecommunications Association, Inc.",
    "SOUTHWEST ARKANSAS TELEPHONE COOPERATIVE, INC.":"Southwest Arkansas Telephone Cooperative, Inc.",
    "South Central Wireless, Inc.":"South Central Wireless Inc.",
    "Southern Montana Telephone Company (SMTC)":"Southern Montana Telephone Company",
    "Steelville Telephone Exchange Inc.":"Steelville Telephone Exchange Inc",
    "Tekstar Communications, Inc. dba Arvig":"Arvig Enterprises, Inc.",
    "Upsala Cooperative Telephone Association dba Sytek":"UPSALA COOPERATIVE TELEPHONE ASSOCIATION",
    "Valley Telephone Cooperative, Inc":"Valley Telephone Cooperative, Inc.",
    "Velocity.Net Communications, Inc.":"Velocity Communications, Inc.",
    "Woodstock Telephone Co.":"Woodstock Telephone Company",
    "ZIRKEL Wireless, LLC":"Zirkel Wireless",
    "coon valley cooperative telephone association inc.":"Coon Valley Co-op Telephone Association, Inc.",
}

bidders["bidder"] = bidders["bidder"].map(lambda x: name_map[x] if x in name_map else x)
bidders

,bidder,t+l_weight,tier,latency
0,"1stel, Inc.",20,Above Baseline,Low
1,4-Corners Consortium,20,Above Baseline,Low
2,AB Indiana LLC,20,Above Baseline,Low
3,ACT,0,Gigabit,Low
4,"AMA Communicaitons, L.L.C.",20,Above Baseline,Low
...,...,...,...,...
311,Yucca Telecom,0,Gigabit,Low
312,Zirkel Wireless,20,Above Baseline,Low
313,"Zito West Holding, LLC",0,Gigabit,Low
314,"Coon Valley Co-op Telephone Association, Inc.",0,Gigabit,Low


In [14]:
collapsed_bids = collapsed_bids.merge(bidders, how="left",left_on="winner", right_on="bidder").drop(columns=["bidder"])
collapsed_bids.to_pickle("../outputs/collapsed_bids.pkl")
collapsed_bids

,GEOID,bidders,all_neighbors,geometry,winner,blocks,round,t+l_weight,tier,latency
0,10010207001,"{'1stel, Inc.': {'neighbors': [], 'round': 9, ...","{10010202002: {('WideOpenWest Finance, LLC', '...","POLYGON ((-86.47038099999999 32.456581, -86.47...",Space Exploration Technologies Corp.,"[10010207001071, 10010207001073, 10010207001095]",13.0,20.0,Above Baseline,Low
1,10010208011,"{'Hughes Network Systems, LLC': {'neighbors': ...","{10010206002: {('WideOpenWest Finance, LLC', '...","POLYGON ((-86.59549799999999 32.381123, -86.59...",Space Exploration Technologies Corp.,"[10010208011010, 10010208011017, 1001020801102...",13.0,20.0,Above Baseline,Low
2,10010208012,"{'Hughes Network Systems, LLC': {'neighbors': ...","{10010201001: {('WideOpenWest Finance, LLC', '...","POLYGON ((-86.591984 32.44164, -86.591976 32.4...",Space Exploration Technologies Corp.,"[10010208012004, 10010208012010, 1001020801201...",13.0,20.0,Above Baseline,Low
3,10010208021,"{'Hughes Network Systems, LLC': {'neighbors': ...","{10010201001: {('WideOpenWest Finance, LLC', '...","POLYGON ((-86.645787 32.554375, -86.6457779999...",RDOF USA Consortium,"[10010208021003, 10010208021006, 1001020802101...",13.0,0.0,Gigabit,Low
4,10010209001,"{'Hughes Network Systems, LLC': {'neighbors': ...","{10010208021: {('Central Access, Inc', 'Centra...","POLYGON ((-86.664761 32.627948, -86.6647069999...",RDOF USA Consortium,"[10010209001000, 10010209001001, 1001020900100...",13.0,0.0,Gigabit,Low
...,...,...,...,...,...,...,...,...,...,...
61736,560399676003,{'AMG Technology Investment Group LLC': {'neig...,"{160199701001: {('Cable One, Inc.', 'Cable ONE...","POLYGON ((-111.045492 43.51067099999999, -111....",Space Exploration Technologies Corp.,"[560399676003075, 560399676003087, 56039967600...",13.0,20.0,Above Baseline,Low
61737,560399677021,{'AMG Technology Investment Group LLC': {'neig...,"{560399676001: {('Rise Broadband', 'Skybeam, L...","POLYGON ((-111.039007 43.472801, -111.036624 4...",Space Exploration Technologies Corp.,"[560399677021047, 560399677021058, 56039967702...",13.0,20.0,Above Baseline,Low
61738,560419753001,{'AMG Technology Investment Group LLC': {'neig...,{490439641011: {('COMCAST CABLE COMMUNICATIONS...,"POLYGON ((-111.046818 41.156019, -111.046818 4...",Space Exploration Technologies Corp.,[560419753001107],13.0,20.0,Above Baseline,Low
61739,560430002001,{'AMG Technology Investment Group LLC': {'neig...,{560039626002: {('Mountain West Technologies C...,"POLYGON ((-108.550788 44.12770099999999, -108....",Space Exploration Technologies Corp.,"[560430002001181, 560430002001513]",13.0,20.0,Above Baseline,Low


In [8]:
def get_number_with_neighbors(col):
    counter = 0
    for bidder, data in col.items():
        if data["neighbor_status"] !=0:
            counter +=1
    return counter

{0, 1, 2, 3, 4, 5, 6, 7}

In [37]:
collapsed_bids["competition bidders"] = collapsed_bids.apply(lambda x: get_number_with_neighbors(x.bidders), axis=1)
collapsed_bids[collapsed_bids["competition bidders"] >=2]
# for k, v in items.items():
#     print(k, v["round"], v["neighbor_status"])

,GEOID,bidders,all_neighbors,geometry,winner,blocks,round,t+l_weight,tier,latency,competition bidders
114,10090501015,{'Charter Communications Inc': {'neighbors': [...,"{10090501011: {('Otelco Inc', 'Otelco Telephon...","POLYGON ((-86.572293 33.820705, -86.5722909999...",Charter Communications Inc,"[10090501015053, 10090501015068, 1009050101507...",15.0,0.0,Gigabit,Low,2
129,10090507003,{'Charter Communications Inc': {'neighbors': [...,"{10090501015: {('COMCAST CABLE COMMUNICATIONS,...","POLYGON ((-86.652458 33.777399, -86.6521559999...",Space Exploration Technologies Corp.,"[10090507003000, 10090507003003, 1009050700300...",13.0,20.0,Above Baseline,Low,2
130,10090507004,{'Charter Communications Inc': {'neighbors': [...,"{10090501015: {('COMCAST CABLE COMMUNICATIONS,...","POLYGON ((-86.64627899999999 33.781403, -86.64...",Space Exploration Technologies Corp.,"[10090507004003, 10090507004007, 1009050700401...",13.0,20.0,Above Baseline,Low,2
203,10150026001,{'Charter Communications Inc': {'neighbors': [...,"{10150015002: {('Cable One, Inc.', 'Cable ONE'...","POLYGON ((-86.146225 33.702176, -86.1457719999...",Charter Communications Inc,"[10150026001000, 10150026001001, 1015002600100...",19.0,0.0,Gigabit,Low,2
204,10150026002,{'Charter Communications Inc': {'neighbors': [...,"{10150025011: {('Cable One, Inc.', 'Cable ONE'...","POLYGON ((-86.06697299999999 33.839278, -86.06...",Charter Communications Inc,"[10150026002000, 10150026002001, 1015002600200...",19.0,0.0,Gigabit,Low,2
...,...,...,...,...,...,...,...,...,...,...,...
61718,560350001012,{'AMG Technology Investment Group LLC': {'neig...,"{560239780001: {('All West/Wyoming, Inc.', 'Al...","POLYGON ((-110.59013 42.448446, -110.59013 42....",Space Exploration Technologies Corp.,"[560350001012019, 560350001012021, 56035000101...",17.0,20.0,Above Baseline,Low,2
61719,560350001021,{'AMG Technology Investment Group LLC': {'neig...,"{560130003002: {('RT', 'Range Telephone Cooper...","POLYGON ((-109.8936 42.865309, -109.893505 42....","Visionary Communications, Inc.","[560350001021000, 560350001021004, 56035000102...",14.0,0.0,Gigabit,Low,2
61720,560350001022,{'AMG Technology Investment Group LLC': {'neig...,"{560130004001: {('Range Telephone Cooperative,...","POLYGON ((-110.53416 43.28313, -110.53392 43.2...","CenturyLink, Inc.","[560350001022009, 560350001022028, 56035000102...",12.0,0.0,Gigabit,Low,2
61721,560379705001,{'AMG Technology Investment Group LLC': {'neig...,"{560239782002: {('LR Communications, Inc.', 'L...","POLYGON ((-110.048908 41.474038, -110.048899 4...",Union Telephone Company,"[560379705001083, 560379705001171, 56037970500...",13.0,35.0,Baseline,Low,2


In [38]:
collapsed_bids["competition bidders"].value_counts()

0    36430
1    17794
2     5627
3     1463
4      349
5       68
6        8
7        2
Name: competition bidders, dtype: int64

In [27]:
processed_bids["neighbor_status"].value_counts()

0    388896
2     24898
1     10337
Name: neighbor_status, dtype: int64

In [29]:
processed_bids[processed_bids["winner"]]["neighbor_status"].value_counts()

0    45433
2     8939
1     2782
Name: neighbor_status, dtype: int64